## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploritory Data Analysis and System Tracking and Monitoring.</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2025">The Repo</a>.  

Once you have updated your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://github.com/apps/databricks">Databricks App on Github</a> or by cloning the repo to your laptop and then uploading the final_project directory and its contents to your workspace using file imports.  Your choice.

<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the this row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches to triggering your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)
- [In class examples - Spark Structured Streaming Performance](https://dbc-f85bdc5b-07db.cloud.databricks.com/editor/notebooks/2638424645880316?o=1093580174577663)

### Be sure your project runs end to end when *Run all* is executued on this notebook! (7 points)

### This project is worth 25% of your final grade.
- DSCC-202 Students have 55 possible points on this project (see points above and the instructions below)
- DSCC-402 Students have 60 possible points on this project (one extra section to complete)

In [0]:
%run ./includes/includes

In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    # Optimize the tables
    optimize_table(BRONZE_DELTA)
    optimize_table(SILVER_DELTA)
    optimize_table(GOLD_DELTA)
    print("Optimized all of the Delta Tables")

In [0]:
%pip install ydata-profiling==4.5.1

In [0]:
!pip install 'pydantic<2'

In [0]:
%pip install transformers

In [0]:
%pip install mlflow

In [0]:
%pip install delta-spark

In [0]:
!pip install 'anyio<4.0.0'

## 1.0 Import your libraries here (2 points)
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

In [0]:
# Import necessary libraries
from pyspark.sql import SparkSession

from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable
import matplotlib.pyplot as plt
import mlflow
import mlflow.pyfunc
import re
import seaborn as sns
import pandas as pd
import numpy as np
from pyspark.sql.window import Window
from pyspark.sql.functions import when, count, sum, min, max

# Define schema for incoming JSON data
json_schema = StructType([
    StructField("date", StringType()),
    StructField("user", StringType()),
    StructField("text", StringType()),
    StructField("sentiment", StringType())
])

spark.sparkContext.defaultParallelism

# Set the number of shuffle partitions based on the cluster size
# For a standard cluster with 8 cores, 32 partitions is a reasonable starting point
# This can be adjusted based on the actual cluster configuration
spark.conf.set("spark.sql.shuffle.partitions", 32)

# Enable Delta Lake features
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

# Configure optimizations for structured streaming
spark.conf.set("spark.sql.streaming.stateStore.providerClass", 
              "org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider")

# Set checkpointing interval for better fault tolerance
spark.conf.set("spark.sql.streaming.checkpointInterval", "10 seconds")

#Setup for Date-Time processing:
spark.conf.set("spark.sql.session.timeZone", "UTC")


print("Libraries imported successfully")

In [0]:
spark.sparkContext.defaultParallelism
spark.conf.set("spark.sql.shuffle.partitions", "32")
spark.conf.get("spark.sql.shuffle.partitions")

num_executors = int(spark.sparkContext.getConf().get("spark.executor.instances", "1"))
cores_per_executor = int(spark.sparkContext.getConf().get("spark.executor.cores", "1"))
total_cores = num_executors * cores_per_executor

print(f"Number of executors: {num_executors}")
print(f" Cores per executor: {cores_per_executor}")
print(f"Total available Spark cores: {total_cores}")


## 2.0 Define and execute utility functions (3 points)
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files

In [0]:

def clear_previous_run():
    """
    Clears all data from previous runs by deleting the delta tables and checkpoint directories
    """
    # Delete the delta tables if they exist
    for delta_path in [BRONZE_DELTA, SILVER_DELTA, GOLD_DELTA]:
        if spark._jvm.org.apache.hadoop.fs.Path(delta_path).getFileSystem(
            spark._jsc.hadoopConfiguration()
        ).exists(spark._jvm.org.apache.hadoop.fs.Path(delta_path)):
            dbutils.fs.rm(delta_path, recurse=True)
            print(f"Deleted delta table at {delta_path}")
    
    # Delete the checkpoint directories if they exist
    for checkpoint_path in [BRONZE_CHECKPOINT, SILVER_CHECKPOINT, GOLD_CHECKPOINT]:
        if spark._jvm.org.apache.hadoop.fs.Path(checkpoint_path).getFileSystem(
            spark._jsc.hadoopConfiguration()
        ).exists(spark._jvm.org.apache.hadoop.fs.Path(checkpoint_path)):
            dbutils.fs.rm(checkpoint_path, recurse=True)
            print(f"Deleted checkpoint directory at {checkpoint_path}")



def stop_all_streams():
    """
    Stops all active streams
    """
    for stream in spark.streams.active:
        try:
            stream.stop()
            print(f"Stopped stream: {stream.name}")
        except:
            print(f"Failed to stop stream {stream.name}")


def optimize_table(delta_path):
    """
    Optimize the Delta table by compacting small files and running VACUUM
    
    Args:
        delta_path (str): Path to the Delta table
    """
    if spark._jvm.org.apache.hadoop.fs.Path(delta_path).getFileSystem(
        spark._jsc.hadoopConfiguration()
    ).exists(spark._jvm.org.apache.hadoop.fs.Path(delta_path)):
        # Optimize the table to merge small files
        spark.sql(f"OPTIMIZE delta.`{delta_path}`")
        print(f"Optimized table at {delta_path}")
        
        # VACUUM to remove files no longer needed
        spark.sql(f"VACUUM delta.`{delta_path}` RETAIN 168 HOURS")
        print(f"Vacuumed table at {delta_path}")
    else:
        print(f"Table at {delta_path} does not exist")


def setup_mlflow_model():
    """
    Set up the MLflow model for sentiment prediction.
    Registers the model if it doesn't exist already.
    
    Returns:
        str: The URI of the model to use for predictions
    """
    try:
        # Check if the model is already registered
        model_version = mlflow.pyfunc.load_model(f"models:/{MODEL_NAME}/Production")
        print(f"Model {MODEL_NAME} is already registered and loaded")
        return f"models:/{MODEL_NAME}/Production"
    except:
        print(f"Model {MODEL_NAME} is not registered, registering it now")
        # Start a new MLflow run
        with mlflow.start_run(run_name="Register HuggingFace Model") as run:
            # Use the huggingface flavor to log the model
            model_info = mlflow.transformers.log_model(
                transformers_model={
                    "model": HF_MODEL_NAME,
                    "task": "text-classification"
                },
                artifact_path="model",
                registered_model_name=MODEL_NAME
            )
            
            # Transition the model to Production
            client = mlflow.tracking.MlflowClient()
            latest_version = client.get_latest_versions(MODEL_NAME, stages=["None"])[0].version
            client.transition_model_version_stage(
                name=MODEL_NAME,
                version=latest_version,
                stage="Production",
            )
            
            print(f"Model {MODEL_NAME} registered with version {latest_version} and transitioned to Production")
            return f"models:/{MODEL_NAME}/Production"


def monitor_metrics(df, metric_name, value_col):
    """
    Create monitoring metrics to track data flow through the pipeline
    
    Args:
        df (DataFrame): DataFrame to collect metrics from
        metric_name (str): Name of the metric to track
        value_col (str): Column to track values from
    """
    count = df.count()
    # Log the count to MLflow
    mlflow.log_metric(f"{metric_name}_count", count)
    
    # Calculate basic statistics if the DataFrame is not empty
    if count > 0 and value_col in df.columns:
        # For numerical columns, calculate statistics
        if df.schema[value_col].dataType.typeName() in ["integer", "long", "double", "float"]:
            stats = df.agg(
                min(col(value_col)).alias("min"),
                max(col(value_col)).alias("max"),
                avg(col(value_col)).alias("avg")
            ).collect()[0]
            
            mlflow.log_metric(f"{metric_name}_{value_col}_min", stats["min"])
            mlflow.log_metric(f"{metric_name}_{value_col}_max", stats["max"])
            mlflow.log_metric(f"{metric_name}_{value_col}_avg", stats["avg"])
    
    print(f"Logged metrics for {metric_name}: {count} records")


def create_schema_visualization(delta_path, table_name):
    """
    Create a visualization of the schema of a Delta table
    
    Args:
        delta_path (str): Path to the Delta table
        table_name (str): Name of the table for display purposes
    """
    if spark._jvm.org.apache.hadoop.fs.Path(delta_path).getFileSystem(
        spark._jsc.hadoopConfiguration()
    ).exists(spark._jvm.org.apache.hadoop.fs.Path(delta_path)):
        # Read the Delta table
        df = spark.read.format("delta").load(delta_path)
        
        # Get the schema
        schema = df.schema
        
        # Create a DataFrame to visualize the schema
        schema_df = pd.DataFrame([
            {"Field": field.name, "Type": str(field.dataType), "Nullable": str(field.nullable)}
            for field in schema
        ])
        
        # Display the schema
        print(f"Schema for {table_name} at {delta_path}:")
        display(schema_df)
    else:
        print(f"Table at {delta_path} does not exist")

In [0]:
%python
# Define the path
TWEET_SOURCE_PATH = "dbfs:/FileStore/tables/raw_tweets/"

def list_source_files(source_path):
    return [f.path for f in dbutils.fs.ls(source_path) if f.path.endswith(".json")]

def count_source_files(file_list):
    return len(file_list)

def preview_file(file_path, num_chars=1000):
    raw = dbutils.fs.head(file_path, num_chars)
    print(f"📄 Preview: {file_path}\n{raw[:num_chars]}")

# Run
source_files = list_source_files(TWEET_SOURCE_PATH)
print(f"✅ Total source files: {count_source_files(source_files)}")
if source_files:
    preview_file(source_files[0])

## 3.0 Transform the Raw Data to Bronze Data using a stream  (8 points)
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using delta lake to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defined in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook

In [0]:
# ENTER YOUR CODE HERE
from pyspark.sql.types import StructType, StringType
from pyspark.sql.functions import input_file_name, current_timestamp


# 1. Define schema for raw JSON data (as per spec)
bronze_schema = StructType() \
    .add("date", StringType()) \
    .add("user", StringType()) \
    .add("text", StringType()) \
    .add("sentiment", StringType())

# 2. Set up a read stream using Auto Loader
raw_stream_df = (
    spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "json")
    .schema(bronze_schema)  # Enforce schema
    .load(TWEET_SOURCE_PATH)
    .withColumn("source_file", input_file_name())
    .withColumn("processing_time", current_timestamp())
)

# 3. Write stream to Bronze Delta table
bronze_stream = (
    raw_stream_df.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", BRONZE_CHECKPOINT)
    .option("mergeSchema", "true")  # Allow new columns if schema evolves
    .queryName("bronze_stream")     # Name the stream
    .start(BRONZE_DELTA)
)

## 4.0 Transform the Bronze Data to Silver Data using a stream (5 points)
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

In [0]:
%python
from pyspark.sql.functions import col, regexp_replace, explode, split, trim, to_timestamp
from pyspark.sql.types import StringType

# ✅ Step 1: Read from Bronze Delta table
bronze_df = (
    spark.readStream
    .format("delta")
    .load(BRONZE_DELTA)
)

# ✅ Step 2: Transform into Silver Data
silver_df = (
    bronze_df
    .filter(col("text").isNotNull() & col("sentiment").isNotNull())
    
    # Convert the date to timestamp
    .withColumn("timestamp", to_timestamp(col("date")))
    
    # Extract mentions as individual rows
    .withColumn("mention", explode(split(col("text"), " ")))  # Split by space
    .filter(col("mention").startswith("@"))  # Keep only tokens that are mentions
    
    # Remove all mentions from the original text to form cleaned_text
    .withColumn("cleaned_text", regexp_replace(col("text"), "@\\w+", ""))
    
    # Clean extra whitespace
    .withColumn("cleaned_text", trim(col("cleaned_text")))
    
    # Select and rename columns to match Silver schema
    .select(
        "timestamp",
        "mention",
        "cleaned_text",
        col("sentiment").alias("Sentiment")
    )
)

# ✅ Step 3: Write to Silver Delta table
silver_stream = (
    silver_df.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", SILVER_CHECKPOINT)
    .queryName("silver_stream")
    .start(SILVER_DELTA)
)

print("🚀 Silver stream started.")

## 5.0 Transform the Silver Data to Gold Data using a stream (7 points)
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

In [0]:
# === Step 1: Load the pretrained model from MLflow ===
import mlflow.pyfunc
model = mlflow.pyfunc.load_model(f"models:/{MODEL_NAME}/Production")

# === Step 2: Required imports ===
from pyspark.sql.functions import col, when, lit
from pyspark.sql.types import StringType, FloatType, StructType, StructField

# === Step 3: Create Spark UDF from MLflow model ===
sentiment_schema = StructType([
    StructField("score", FloatType()),
    StructField("label", StringType())
])

@udf(sentiment_schema)
def sentiment_predict_udf(text):
    import pandas as pd
    try:
        if not text or not isinstance(text, str):
            return (0.0, "UNKNOWN")
        df = pd.DataFrame([text], columns=["text"])
        result = model.predict(df)
        return (float(result.iloc[0]["score"]) * 100, result.iloc[0]["label"].upper())
    except Exception:
        return (0.0, "ERROR")

# === Step 4: Start Silver stream ===
silver_stream = (
    spark.readStream
         .format("delta")
         .load(SILVER_DELTA)
)

# === Step 5: Apply UDF and build Gold schema ===
gold_ready = (
    silver_stream
        .withColumn("prediction", sentiment_predict_udf(col("cleaned_text")))
        .withColumn("predicted_score", col("prediction.score"))
        .withColumn("predicted_sentiment", col("prediction.label"))
        .withColumn("sentiment_id", when(col("sentiment").isin("negative", "NEG"), 0).otherwise(1))
        .withColumn("predicted_sentiment_id", when(col("predicted_sentiment").isin("NEGATIVE", "NEG"), 0).otherwise(1))
        .select(
            col("timestamp"),
            col("mention"),
            col("cleaned_text"),
            col("sentiment"),
            col("predicted_score"),
            col("predicted_sentiment"),
            col("sentiment_id"),
            col("predicted_sentiment_id")
        )
)

# === Step 6: Write to Gold Delta Table as a Stream ===
gold_output_stream = (
    gold_ready.writeStream
              .queryName("gold_stream")
              .format("delta")
              .option("checkpointLocation", GOLD_CHECKPOINT)
              .outputMode("append")
              .start(GOLD_DELTA)
)

print("✅ Gold stream started with correct schema.")


## 6.0 Monitor your Streams (5 points)
- Setup a loop that runs at least every 10 seconds
- Print a timestamp of the monitoring query along with the list of streams, rows processed on each, and the processing time on each
- Run the loop until all of the data is processed (0 rows read on each active stream)
- Plot a line graph that shows the data processed by each stream over time
- Plot a line graph that shows the average processing time on each stream over time

In [0]:
# List all active streams
for stream in spark.streams.active:
    print(f"🔄 Stream Name: {stream.name}")
    print(f"  Is Active: {stream.isActive}")
    print(f"  Status: {stream.status['message']}")
    print("-" * 40)


In [0]:
import time
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt

# Initialize tracking
monitoring_data = []
idle_checks = 0
max_idle_checks = 3  # allow 3 idle loops before stopping
check_interval = 10  # seconds between checks

print("📡 Monitoring active streams...")
while True:
    active_queries = spark.streams.active
    timestamp = datetime.now().strftime("%H:%M:%S")

    all_idle = True

    for query in active_queries:
        progress = query.lastProgress
        if progress:
            num_input_rows = progress["numInputRows"]
            proc_time = float(progress["durationMs"].get("addBatch", 0)) / 1000
            name = query.name

            print(f"[{timestamp}] Stream: {name} | Rows: {num_input_rows} | Time: {proc_time:.2f}s")

            monitoring_data.append({
                "timestamp": timestamp,
                "stream": name,
                "rows": num_input_rows,
                "time": proc_time
            })

            if num_input_rows > 0:
                all_idle = False

    if all_idle:
        idle_checks += 1
        print(f"No data processed. Idle check {idle_checks}/{max_idle_checks}")
    else:
        idle_checks = 0

    if idle_checks >= max_idle_checks:
        print("Streams idle for 3 consecutive checks. Stopping monitoring.")
        break

    time.sleep(check_interval)


In [0]:
spark.read.format("delta").load(BRONZE_DELTA).count()
spark.read.format("delta").load(SILVER_DELTA).count()
spark.read.format("delta").load(GOLD_DELTA).count()

In [0]:
import matplotlib.pyplot as plt
import pandas as pd

# Make sure `monitoring_data` is a list of dictionaries with keys: 'timestamp', 'stream', 'rows', 'time'
df_monitor = pd.DataFrame(monitoring_data)

# Ensure timestamp is in datetime format (if it's not already)
df_monitor["timestamp"] = pd.to_datetime(df_monitor["timestamp"])

# Plot 1: Rows processed over time
plt.figure(figsize=(12, 5))
for stream in df_monitor["stream"].unique():
    subset = df_monitor[df_monitor["stream"] == stream]
    plt.plot(subset["timestamp"], subset["rows"], marker="o", label=stream)

plt.title("Rows Processed Over Time")
plt.xlabel("Time")
plt.ylabel("Number of Rows")
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [0]:
# Plot 2: Processing time over time
plt.figure(figsize=(12, 5))
for stream in df_monitor["stream"].unique():
    subset = df_monitor[df_monitor["stream"] == stream]
    plt.plot(subset["timestamp"], subset["time"], marker="s", label=stream)

plt.title("Average Processing Time Over Time")
plt.xlabel("Time")
plt.ylabel("Processing Time (s)")
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## 7.0 Bronze Data Exploratory Data Analysis (5 points)
- How many tweets are captured in your Bronze Table?
- Are there any columns that contain Nan or Null values?  If so how many and what will you do in your silver transforms to address this?
- Count the number of tweets by each unique user handle and sort the data by descending count.
- How many tweets have at least one mention (@) how many tweet have no mentions (@)
- Plot a bar chart that shows the top 20 tweeters (users)


In [0]:
import matplotlib.pyplot as plt
import pandas as pd
from pyspark.sql.functions import col

# Load Bronze table
bronze_df = spark.read.format("delta").load(BRONZE_DELTA)

# 1. Count total tweets
total_tweets = bronze_df.count()
print(f"Total tweets in Bronze Table: {total_tweets}")

# 2. Null value analysis
null_counts = bronze_df.select([col(c).isNull().cast("int").alias(c) for c in bronze_df.columns]) \
    .groupBy().sum().collect()[0].asDict()
print("Null counts in each column:", null_counts)

# 3. Count tweets per user
user_counts = bronze_df.groupBy("user").count().orderBy("count", ascending=False)
top_20_users_df = user_counts.limit(20).toPandas()

# 4. Mentions analysis
mention_stats = bronze_df.selectExpr(
    "CASE WHEN text LIKE '%@%' THEN 'has_mention' ELSE 'no_mention' END AS mention_status"
).groupBy("mention_status").count().toPandas()

print("Mentions Summary:")
print(mention_stats)

# 5. Plot top 20 tweeters
plt.figure(figsize=(12, 6))
plt.bar(top_20_users_df["user"], top_20_users_df["count"], color="steelblue")
plt.xticks(rotation=90)
plt.title("Top 20 Tweeters by Tweet Count")
plt.xlabel("User")
plt.ylabel("Tweet Count")
plt.tight_layout()
plt.show()


## 8.0 Capture the accuracy metrics from the gold table in MLflow  (4 points)
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the model name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

In [0]:
%python
import mlflow
from mlflow import MlflowClient
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Load Gold Delta table into Pandas for evaluation
gold_df = spark.read.format("delta").load(GOLD_DELTA).toPandas()

# Drop rows with missing predictions or labels
gold_df = gold_df.dropna(subset=["sentiment_id", "predicted_sentiment_id"])

# Convert to integer types
y_true = gold_df["sentiment_id"].astype(int)
y_pred = gold_df["predicted_sentiment_id"].astype(int)

# Compute metrics
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)
conf_matrix = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["NEG", "POS"], yticklabels=["NEG", "POS"])
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
conf_matrix_path = "/tmp/confusion_matrix.png"
plt.savefig(conf_matrix_path)
plt.close()

# Get MLflow model version
client = MlflowClient()
model_version = client.get_latest_versions(MODEL_NAME, stages=["Production"])[0].version

# Get Delta Table version
from delta.tables import DeltaTable
silver_version = DeltaTable.forPath(spark, SILVER_DELTA).history().select("version").first()["version"]

# Log to MLflow
with mlflow.start_run(run_name="Gold Evaluation Metrics"):
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)

    mlflow.log_param("model_name", MODEL_NAME)
    mlflow.log_param("model_version", model_version)
    mlflow.log_param("silver_table_version", silver_version)

    mlflow.log_artifact(conf_matrix_path)

print("Evaluation complete and metrics logged to MLflow.")

Gold Evaluation Metrics
![](https://drive.google.com/file/d/1flI_5YIRSEPcNqXblTR63If43Co6sEAu/view?usp=drive_link)

## 9.0 Application Data Processing and Visualization (6 points)
- How many mentions are there in the gold data total?
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
from pyspark.sql.functions import col, when, min, max

# Reload the gold table
gold_df = spark.read.format("delta").load(GOLD_DELTA)

# Normalize predicted sentiment values
gold_df = (
    gold_df
    .withColumn("predicted_sentiment", when(col("predicted_sentiment").isin("POS", "positive"), "positive")
                .when(col("predicted_sentiment").isin("NEG", "negative"), "negative")
                .when(col("predicted_sentiment").isin("NEU", "neutral"), "neutral")
                .otherwise("unknown"))
)

# Aggregate mention stats
mention_stats_df = (
    gold_df.groupBy("mention")
        .agg(
            min("timestamp").alias("min_timestamp"),
            max("timestamp").alias("Max_timestamp"),
            (sum(when(col("predicted_sentiment") == "negative", 1).otherwise(0))).alias("negative"),
            (sum(when(col("predicted_sentiment") == "neutral", 1).otherwise(0))).alias("neutral"),
            (sum(when(col("predicted_sentiment") == "positive", 1).otherwise(0))).alias("positive")
        )
        .withColumn("total", col("negative") + col("neutral") + col("positive"))
        .filter(col("total") > 0)
)

# Cache for reuse
mention_stats_df.cache()

# Show sample
mention_stats_df.show(10, truncate=False)




In [0]:
top_positive_df = mention_stats_df.orderBy(col("positive").desc()).limit(20).toPandas()

import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.bar(top_positive_df["mention"], top_positive_df["positive"], color="green")
plt.xticks(rotation=90)
plt.title("Top 20 Mentions with Positive Sentiment")
plt.xlabel("Mention")
plt.ylabel("Positive Tweet Count")
plt.tight_layout()
plt.show()


In [0]:
top_negative_df = mention_stats_df.orderBy(col("negative").desc()).limit(20).toPandas()

plt.figure(figsize=(12, 6))
plt.bar(top_negative_df["mention"], top_negative_df["negative"], color="red")
plt.xticks(rotation=90)
plt.title("Top 20 Mentions with Negative Sentiment")
plt.xlabel("Mention")
plt.ylabel("Negative Tweet Count")
plt.tight_layout()
plt.show()


## 10.0 Clean up and completion of your pipeline (3 points)
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook. Note: In the includes there is a variable START_TIME that captures the starting time of the notebook.

In [0]:
import time
from pyspark.sql.functions import col
from datetime import timedelta

# 1.Check and print all active streams
print("Active Streams:")
for query in spark.streams.active:
    print(f"• Stream Name: {query.name}, ID: {query.id}, Is Active: {query.isActive}")

# 2.Stop all active streams
print("\n⏹Stopping all active streams...")
for query in spark.streams.active:
    print(f"→ Stopping stream: {query.name} (ID: {query.id})")
    query.stop()

print("All streams have been stopped.")

# 3.Compute elapsed time
try:
    elapsed_seconds = int(time.time() - START_TIME)
    elapsed_time = str(timedelta(seconds=elapsed_seconds))
    print(f"\n⏱ Notebook Elapsed Time: {elapsed_time}")
except NameError:
    print("START_TIME not defined. Please make sure it was initialized at notebook start.")


## 11.0 How Optimized is your Spark Application (Grad Students Only) (5 points)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

 
I’ve included screenshots from a few job runs based on the code above, and here are some key takeaways from what I observed.

Spill

![](https://drive.google.com/file/d/1BOH32gVQlsN9Ksa775QL8pH_Si9Unc7d/view?usp=drive_link)

![](https://drive.google.com/file/d/1sNJEduvBGitM6Rx1915mavRkQay9OFzT/view?usp=drive_link)
The system ran very efficiently—there were no memory issues, no garbage collection needed, and barely any data shuffling (only 615 bytes). The task finished in just 7 milliseconds, showing it executed smoothly in memory without any slowdowns.

 

Skew

![](https://drive.google.com/file/d/1jk4gl9_rHHAicV0voHZD6JyFHSPwOeVZ/view?usp=drive_link)

I checked the Stage Details and saw that all tasks ran quickly (e.g., 38–81 ms) and were evenly distributed—8 out of 8 tasks finished successfully in one stage.

The input and shuffle data were evenly spread out (e.g., 161.4 KiB per task), with no task taking noticeably longer or processing more data. Overall, the job showed no signs of skew and ran efficiently with balanced workloads and good parallelism.

Shuffle

From the Spark UI, shuffle activity was minimal (520 B to 161.4 KiB), tasks finished quickly (under 100 ms), and no stages had issues. The number of shuffle partitions and data movement were well within limits. Overall, the pipeline ran efficiently with no shuffle bottlenecks—likely thanks to narrow transformations or smart optimizations like broadcast joins or partition pruning.

Storage

![](https://drive.google.com/file/d/1lXT5Cy_bxrCeWekoCfXOLf26CAM7aivp/view?usp=drive_link)

The queries ran fast—most under 2 seconds—even for typically slower operations like .orderBy() and .distinct(). This suggests the Delta tables are well-optimized, likely stored in Parquet format with minimal small files or metadata overhead, making them easy and quick for Spark to read.


Serialization

![](https://drive.google.com/file/d/1T-5eh6ujisOuxH_gpYJ5DcBbbrssJgj0/view?usp=drive_link)
 
Result Serialization Time is minimal or absent in the task bar. The majority of the task time is spent in Executor Computing and Task Deserialization, with Result Serialization contributing very little.
